In [1]:
import numpy as np
import pandas as pd

import sys
import os

sys.path.append(os.path.abspath('..'))
from utils.db import get_db

In [2]:
engine = get_db()

In [21]:
query = """select sale_id,  total_sales, qty
from sales
where store_number='440'
and sale_date between '5/15/2025' and '5/15/2025'
and sale_type='Sale'
and item_ring_type='ITEM'
order by sale_date, sale_id"""

In [22]:
ds = pd.read_sql(query, engine)

2025-06-13 20:53:48,974 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-13 20:53:48,975 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-06-13 20:53:48,975 INFO sqlalchemy.engine.Engine [cached since 310.4s ago] {'table_name': "select sale_id,  total_sales, qty\nfrom sales\nwhere store_number='440'\nand sale_date between '5/15/2025' and '5/15/2025'\nand sale_type='Sale'\nand item_ring_type='ITEM'\norder by sale_date, sale_id", 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-06-13 20:53:49,032 INFO sqlalche

In [23]:
ds.tail(20)

,sale_id,total_sales,qty
5546,479786,2.58,1.0
5547,479786,2.99,1.0
5548,479786,2.99,1.0
5549,479786,1.89,1.0
5550,479787,0.70,1.0
5551,479787,2.00,1.0
5552,479787,6.49,1.0
5553,479787,1.59,1.0
5554,479787,7.99,1.0
5555,479787,1.59,1.0


In [24]:
grouped = ds.groupby(['sale_id']).agg(
    total_sales=('total_sales', 'sum'),
    qty=('qty', 'sum'),
).reset_index()

In [25]:
grouped

,sale_id,total_sales,qty
0,35238,5.99,1.0
1,102677,75.00,1.0
2,102679,32.91,1.0
3,102680,40.00,1.0
4,102681,58.46,1.0
...,...,...,...
866,479783,64.68,16.0
867,479784,12.47,5.0
868,479785,53.93,9.0
869,479786,30.08,12.0


In [26]:
grouped['anomaly_flag'] = (
    (grouped['total_sales'] > 500) |
    (grouped['qty'] > 50) 
)

In [28]:
mask1 = grouped['total_sales'] > 500
mask2 = grouped['qty'] > 50
grouped[mask1 | mask2]

,sale_id,total_sales,qty,anomaly_flag
40,106294,158.94,51.0,True
620,374537,181.42,51.0,True
672,479581,201.55,53.0,True
833,479750,241.83,53.0,True
